<a href="https://colab.research.google.com/github/takatakamanbou/ML/blob/2024/ML2024_ex05notebookB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ML ex05notebookB

<img width=72 src="https://www-tlab.math.ryukoku.ac.jp/~takataka/course/ML/ML-logo.png"> [この授業のウェブページ](https://www-tlab.math.ryukoku.ac.jp/wiki/?ML/2024)


----
## ニューラルネットワークと深層学習 (1)
----

**ニューラルネットワーク**(Neural Network)は，もともとは，ヒトなどの生物の「ニューロン」（神経細胞，Neuron）が互いに繋がり合って作る「回路網」(Network)を指す言葉です（注）．
もともとは，神経細胞や脳の仕組みをまねた数理モデルという色合いがありましたが，近年ではその色は薄れ，実用的な機械学習モデルとして発展してきています．
このようなニューラルネットワークの学習は **深層学習**(Deep Learning) と呼ばれ，いわゆる人工知能(AI)の中核的な技術となっています．何が「深層」，「Deep」なのかは後述します．回帰にも識別にも使うことができます．

<span style="font-size: 75%">
※注: 日本語では「神経回路網」と言います．生体内の本物の神経回路網と区別するために，「人工」(Artifitial)を付けて「人工神経回路網」(Artifitial Neural Netrowk)と呼ぶこともあります．
</span>




----
### 準備



以下，コードセルを上から順に実行してながら読んでいってね．

In [ ]:
# 準備あれこれ
import numpy as np
import matplotlib.pyplot as plt
import seaborn
seaborn.set()

# scikit-learn のいろいろ
from sklearn.datasets import make_moons
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier

---
### 線形モデルの限界

回帰のための機械学習手法である線形回帰や，識別のための機械学習手法であるロジスティック回帰は，モデルの構造が簡単である分，性能に限界があります．

#### 線形回帰

$D$次元入力の線形回帰モデルは，次の式で表されます．

$$
f(\mathbf{x}) = w_0 + w_1x_1 + w_2x_2 + \cdots + w_Dx_D
$$

この式は，$\mathbf{x}$ と出力の正解が作る $(D+1)$ 次元空間の中で $D$次元の平面（超平面）を表します．
そのため，平面にうまく当てはまらないようなデータではよい予測ができません．
以下に例を図示します．

<img src="https://www-tlab.math.ryukoku.ac.jp/~takataka/course/ML/planefitting2.png" width="75%">



上の図の青い点 $(x_1, x_2, y)$ は学習データを表し，赤いメッシュは線形回帰モデルを学習させて得られた平面を表します．
一方，青いメッシュは学習データの点が乗っている曲面，すなわち，$(x_1, x_2)$ と $y$ の間の真の関係を表します．

左の場合，学習データがほぼ平面に乗っているため，線形回帰モデルは真の関係をうまくとらえられています．
しかし，右の場合，データが非線形な性質を持つため，線形回帰モデルは学習データの真の関係をとらえきれておらず，予測がうまくいきません．

#### ロジスティック回帰

$D$ 次元入力を 2 クラスに分類するロジスティック回帰モデルは，次の通りです．

$$
\begin{aligned}
f(\mathbf{x}) &= \sigma(w_0 + w_1x_1+\cdots + w_Dx_D) = \sigma\left(w_0 + \sum_{d=1}^{D}w_dx_d \right) \\
&= \frac{1}{1+\exp{\left( - \left( w_0 + \sum_{d=1}^{D}w_dx_d \right) \right)}}
\end{aligned}
$$

出力 $f(\mathbf{x})$ は $0 < f(\mathbf{x}) < 1$ であり， このモデルは，$f(\mathbf{x}) = \frac{1}{2}$ を境界として入力 $\mathbf{x}$ を2つのクラスに分けています．

$$
f(\mathbf{x}) = \frac{1}{2} \Leftrightarrow w_0 + \sum_{d=1}^{D} w_dx_d = 0
$$

ですから，2つのクラスの境界は，平面（$D$次元空間中の $(D-1)$次元超平面，$D=2$のときは直線）となります．クラス数が3以上の場合の説明は省略しますが，そのときも2つのクラスの間の境界はやはり平面となります．



このように，ロジスティック回帰モデルは，クラスとクラスの間を平面でしか分けられないため，クラスの境界がより複雑な場合には予測があまりうまくいきません．
そのような問題の例を以下に示します．



In [ ]:
# 線形モデルではうまく識別できない例
moonX, moonY = make_moons(n_samples=200, noise=0.25, random_state=4649)

fig, ax = plt.subplots()
ax.scatter(moonX[moonY==0, 0], moonX[moonY==0, 1])
ax.scatter(moonX[moonY==1, 0], moonX[moonY==1, 1])
ax.set_xlim(-1.5, 2.5)
ax.set_ylim(-2, 2)
ax.set_aspect('equal')
plt.show()

上記の2次元2クラス識別の問題をロジスティック回帰で解いてみると...

ここでは，Python のための機械学習ライブラリのひとつである [scikit-learn](https://scikit-learn.org/) の中の [sklearn.linear_model.LogisticRegression](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html) を用いています．

In [ ]:
# ロジスティック回帰
model = LogisticRegression()
model.fit(moonX, moonY)
yy = model.predict(moonX)
cc = np.sum(yy == moonY)
print(f'正答率: {cc}/{len(yy)} = {cc/len(yy):.3f}')

# グラフ描画用のグリッドデータの作成
K = 2
xmin, xmax = -1.5, 2.5
ymin, ymax = -2, 2
npoints = 100
dx, dy = (xmax - xmin)/npoints, (ymax - ymin)/npoints
x_mesh, y_mesh = np.mgrid[xmin:xmax:dx, ymin:ymax:dy]
X_mesh = np.dstack((x_mesh, y_mesh))

# X_mesh の各点における事後確率の推定
p = model.predict_proba(X_mesh.reshape((-1, 2)))
pp = p.reshape((X_mesh.shape[0], X_mesh.shape[1], K))

# グラフの描画
fig, ax = plt.subplots()
cmap = ['Blues', 'Oranges', 'Greens']
for k in range(K):
    ax.scatter(moonX[moonY==k, 0], moonX[moonY==k, 1])
    ax.contourf(x_mesh, y_mesh, pp[:, :, k], levels=[0.5, 0.6, 0.7, 0.8, 0.9, 1.0], cmap=cmap[k], alpha=0.3)
ax.set_xlim(xmin, xmax)
ax.set_ylim(ymin, ymax)
ax.set_aspect('equal')
plt.show()

この図では，モデル出力 $f(\mathbf{x})$ が $0$ に近い領域と $1$ に近い領域をオレンジと青で塗り分けてあります（白いところが両者の境界）．
ロジスティック回帰モデルでは2クラスを平面で（この場合データが2次元なので直線で）しか分けられないため，適切な識別ができていないことが分かります．

---
### 階層型ニューラルネットワーク

機械学習の研究分野では，線形回帰やロジスティック回帰のモデルの限界を超えるために，様々な改良が考えられてきました．
ここで紹介する **階層型ニューラルネットワーク** は，そのような改良手法のひとつです．
階層型ニューラルネットワークでは，入力から出力へと至る計算過程を複数の **層** (layer)を直列につなげた形で構成し，途中の層で入力を非線形変換することで，非線形な回帰や識別を実現します．

<img width="60%" src="https://www-tlab.math.ryukoku.ac.jp/~takataka/course/ML/neuralnet1.png" align="right">

図の左側は，2次元のデータを3クラスに分類するロジスティック回帰モデルを模式的に表したものです．
図の「出力層」（薄く色づけされた縦長の長方形）の中の3つの○は，このモデルの出力 $\hat{y}_1, \hat{y}_2, \hat{y}_3 $ を表しています．
これらの値は，入力 $x_1, x_2$ （「入力」の所にある小さい○） から1段階の計算で求まります．

一方，図の右側は，入力と出力の間に層をひとつ挟んだ階層型ニューラルネットワークの例です．
入力と出力層の間に挟まれる層を，**中間層** (middle layer)または **隠れ層** (hidden layer) といいます．中間層と出力層にある○は **ニューロン** (neuron，神経細胞のこと)と呼ばれるもので，階層型ニューラルネットにおける計算の単位です．

この例では，2つの入力が中間層の4つのニューロンに渡されて4つの値が計算され，その値が出力層の3つのニューロンに渡されて3つの出力が計算されます．
階層型ニューラルネットワークは，このように層ごとに段階的に計算を行う構造をもち，ある層の値から次の層の値を求める計算に非線形性を持たせることで，複雑な入出力関係を表すことができます．



<img width="40%" src="https://www-tlab.math.ryukoku.ac.jp/~takataka/course/ML/neuralnet2.png" align="right">

右図は，階層型ニューラルネットワークの例をもっと具体的に示したものです．このニューラルネットワークの入出力をきちんと式で表してみましょう．

このニューラルネットワークが$D$次元の入力を受け取ると，それらはまず中間層にある $H$ 個のニューロンへと伝わります．
中間層の $h$ 番目のニューロンは，次の式にしたがって自身の値 $y_h$ を計算します．

$$
y_h = \sigma\left( v_{h,0} + \sum_{d=1}^{D}v_{h,d}x_d\right) \qquad (H = 1, 2, \ldots, H)
$$

このニューロンは， $v_{h,0}$ から $v_{h,D}$ までの $D+1$ 個のパラメータ（注）を持っています（図の青い線はその一つ）．
$\sigma$ は **活性化関数** (activation function) と呼ばれる非線形変換です（詳しくは後述します）．


次に，これら中間層ニューロンの値が出力層にある $M$ 個のニューロンへと伝わります．
出力層の $m$ 番目のニューロンは，次の式にしたがって自身の値 $z_m$ を計算します．

$$
z_m = \sigma\left( w_{m,0} + \sum_{h=1}^{H}w_{m,h}y_h\right)\qquad (M = 1, 2, \ldots, M)
$$

このニューロンは， $w_{m,0}$ から $w_{m,H}$ までの $H+1$ 個のパラメータを持っています（図の緑の線はその一つ）．
これら出力層ニューロンの値がニューラルネットワークの出力となります．

このニューラルネットワークの場合，学習によって調節されるパラメータは $v_{h,d}$ ($h = 1, 2, \ldots, H, d = 0, 1, 2, \ldots, D$) が $H\times(D+1)$ 個と $w_{m,h}$ ($m=1,2,\ldots,M, h = 1, 2, \ldots, H$) が $M\times(H+1)$ の計 $H(D+1)+M(H+1)$ 個あります．
これらは通常教師あり学習によって調節されます．その方法については後述します．

ここでは中間層が一つだけの例を示しましたが，2層以上の中間層を用いることも可能です．

※注: ニューラルネットワークのパラメータのことを **重み** (weight) や **結合重み**  (connection weight) ということもあります．



ニューラルネットワークの活性化関数は，ニューロンの入出力に非線形性をもたせる役割を果たします．
活性化関数としては，ロジスティック回帰モデルでも使われている**シグモイド関数**(sigmoid function)の他に，**双曲線正接関数**(hyperbolic tangent function) や **Rectified Linear 関数**（**ReLU関数**ともいいます）などがよく用いられます（注）．

$$
\begin{aligned}
\mbox{Logistic Sigmoid}\quad  & \sigma(s) = \frac{1}{1+{\rm e}^{-s}}\\
\mbox{Hyperbolic Tangent}\quad & \sigma(s) = {\rm tanh}(s) = \frac{{\rm e}^{s} - {\rm e}^{-s}}{{\rm e}^{s} + {\rm e}^{-s}}\\
\mbox{Rectified Linear}\quad & \sigma(s) = \left\{
    \begin{array}{ll}
    s & (s \ge 0)\\
    0 & {\rm otherwise}\\
\end{array} \right.
\end{aligned}
$$

以下にそれぞれのグラフを示します．

<span style="font-size: 75%">
※注: 「シグモイド」は「S字状の」という意味の語なので，双曲線正接関数なども含めて類似した形をしている関数たちの総称です．式(X)のものは，正確には「ロジスティックシグモイド関数」(logistic sigmoid function)といいます．
また，「Rectified Linear」な活性化関数を採用したニューロンを Rectified Linear Unit と呼ぶことから，この活性化関数を「ReLU」と呼ぶことがあります．
</span>

In [ ]:
# 活性化関数の値を計算
xmin, xmax = -4, 4
X = np.linspace(xmin, xmax, num=100)
Y1 = 1/(1+np.exp(-X)) # ロジスティックシグモイド
Y2 = np.tanh(X)       # 双曲線正接
Y3 = np.maximum(X, 0) # ReLU

# グラフに描く
fig = plt.figure(facecolor='white', figsize=(10,2.5))
ax1 = fig.add_subplot(131)
ax1.set_xlim(xmin, xmax)
ax1.set_ylim(-1.2, 1.2)
ax1.axhline(y=0, color='black', linestyle='-')
ax1.axvline(x=0, color='black', linestyle='-')
ax1.axhline(y=1, color='gray', linestyle='--')
ax1.plot(X, Y1, c='red', linewidth=2, label='Sigmoid')
ax1.legend()
ax2 = fig.add_subplot(132)
ax2.set_xlim(xmin, xmax)
ax2.set_ylim(-1.2, 1.2)
ax2.axhline(y=0, color='black', linestyle='-')
ax2.axvline(x=0, color='black', linestyle='-')
ax2.axhline(y=1, color='gray', linestyle='--')
ax2.axhline(y=-1, color='gray', linestyle='--')
ax2.plot(X, Y2, c='red', linewidth=2, label='tanh(s)')
ax2.legend()
ax3 = fig.add_subplot(133)
ax3.set_xlim(xmin, xmax)
ax3.set_ylim(xmin, xmax)
ax3.axhline(y=0, color='black', linestyle='-')
ax3.axvline(x=0, color='black', linestyle='-')
ax3.plot(X, Y3, c='red', linewidth=2, label='ReLU')
ax3.legend()
plt.show()

上述の例では中間層が一つだけでしたが，中間層を2つ以上持つ階層型ニューラルネットワークも作ることができます．
中間層を増やせば，より複雑な非線形変換が可能となります．しかし，多くの中間層から成るニューラルネットワークの学習は難しく，計算コストもかかるため，以前（20世紀末ころ）は数層程度が限界でした．それが，研究の進展と，計算機の能力の飛躍的な向上により，近年（2010年ころ以降）では数十層以上あるニューラルネットワークの学習が可能となり，実用レベルの性能が得られるようになりました（注）．
そのような層の多いニューラルネットワークを **深層ニューラルネットワーク** (Deep Neural Network) といい，深層ニューラルネットワークを用いる機械学習を **深層学習** (Deep Learning) といいます．

<span style="font-size: 75%">
※注: インターネットの普及により，データを容易にたくさん集められるようになって，大規模なデータで学習させることが可能になった，というのも大きな原動力になりました．
</span>



---
### ［補足］ ニューロン？ニューラルネットワーク？


<img width="40%" src="https://www-tlab.math.ryukoku.ac.jp/~takataka/course/ML/neuron3.png" align="right">


ヒトの脳には数百億のニューロンが存在しています．
ニューロンは互いに信号を伝達する機能をもっており，多数のニュー
ロンが相互につながりあって情報処理を行なっています．
生命維持から感情・思考にいたるまでの脳機能は，主にこれらニューロ
ンの集団が担っているものと考えられています．

右図の上段は，実際のニューロンを模式的に描いたもの，中段は，これらニューロン間の信号伝達の様子を表したものです．大きな丸がニューロンを表し，線分がニューロン間のつながりを表します．

ニューロンのふるまいを思い切って単純化すると，次のようにまとめられます．

1. 他のニューロンの出力を受け取る
1. それらの和を計算する．ただし，ニューロン間のつながりの強さに応じて，他のニューロンからやってきた値を重み付けした和を計算する．
1. その重み付け和の値に応じて自分の値を決めて出力する．

このようなニューロンのふるまいは，次式のようにモデル化できます．

$$
y = \sigma\left(w_0 + \sum_{d=1}^{D}w_dx_d\right) \qquad (1)
$$

ただし，$x_d$ は他のニューロンの出力，$y$ は自分の出力です．

$w_d$ は $d$ 番目のニューロンと自分との間のつながりの強さを決める量で，「結合重み」(connection weight)や「重み」(weight)などと呼ばれます．$w_0$は「しきい値」(threshold)または「バイアス項」(bias term)などと呼ばれます．

$\sigma$は，重み付け和の値を変換する役割をする関数です．活性化関数(activation function)と呼ばれます．

ニューロンの出力は，他のニューロンからやってくる値が同じでも，重みの値が変わると変化します．つまり，重みがパラメータとなっています．実際の脳内でも，シナプス（神経細胞同士の信号伝達を橋渡しする役割をする）での信号の伝わり方が変化することで記憶や学習が起こっていると考えられています．


<img width="60%" src="https://www-tlab.math.ryukoku.ac.jp/~takataka/course/ML/neuralnet.png" align="right">

ニューラルネットワークとは，上述のようなニューロンが相互に繋がりあったものです．
その繋がり方としては，図の左のように相互に無秩序に繋がるようなものも考えられるのですが，近年のAIで幅広く採用されているのは，上述した階層型ニューラルネットワークです（図右）．


階層型ニューラルネットワークでは，情報が入力されると，層から層へと順次伝わっていき，最終的な出力がなされます．
このように情報の伝わり方が一方向であることから，「フィードフォワード（feed-forward, 順伝播）型」と呼ぶこともあります（注）．

<span style="font-size: 75%">
※注: 「多層パーセプトロン」(Multi-Layer Perceptron)という呼び方もあります．
</span>

---
### 例: ニューラルネットワークによる2次元データの2クラス識別

「線形モデルの限界」の節に出てきた，ロジスティック回帰ではうまく識別できない例をニューラルネットワークに識別させてみましょう．
ネットワークの構造の詳細や学習の方法については，後の回で説明します．

次のコードセルの選択肢は，それぞれ次のモデルを表します：
- `logistic`: ロジスティック回帰モデル．
- `2-layer (2000,)`: 入力-中間層-出力層という構造のニューラルネット（2層ニューラルネット）．中間層のニューロン数は2000．
- `3-layer (1000, 1000)`: 入力-中間層-中間層-出力層という構造のニューラルネット（3層ニューラルネット）．2つの中間層のニューロン数はいずれも1000．

ニューラルネットワークはロジスティック回帰よりも複雑なモデルなので，学習に少し時間がかかります．

ここでは，Python のための機械学習ライブラリのひとつである [scikit-learn](https://scikit-learn.org/) の中の [sklearn.neural_network.MLPClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPClassifier.html) を用いています．



In [ ]:
#@title デモ: 学習させるモデルを選んでね
option = '3-layer (1000, 1000)' #@param ['logistic', '2-layer (2000,)',  '3-layer (1000, 1000)'] {allow-input: false}

if option == 'logistic':
    model = LogisticRegression()
else:
    if option == '2-layer (2000,)':
        numNeurons = (2000, )
    elif option == '3-layer (1000, 1000)':
        numNeurons = (1000, 1000, )
    model = MLPClassifier(hidden_layer_sizes=numNeurons, activation='relu', verbose=False)

moonX, moonY = make_moons(n_samples=200, noise=0.25, random_state=4649)
model.fit(moonX, moonY)
yy = model.predict(moonX)
cc = np.sum(yy == moonY)
print()
print(f'{option}   正答率: {cc}/{len(yy)} = {cc/len(yy):.3f}')

次のセルを実行すると，上で学習させたモデルが入力の2次元平面をどのように2クラスに分けるかを可視化します．

In [ ]:
# グラフ描画用のグリッドデータの作成
K = 2
xmin, xmax = -1.5, 2.5
ymin, ymax = -2, 2
npoints = 100
dx, dy = (xmax - xmin)/npoints, (ymax - ymin)/npoints
x_mesh, y_mesh = np.mgrid[xmin:xmax:dx, ymin:ymax:dy]
X_mesh = np.dstack((x_mesh, y_mesh))

# X_mesh の各点における事後確率の推定
p = model.predict_proba(X_mesh.reshape((-1, 2)))
pp = p.reshape((X_mesh.shape[0], X_mesh.shape[1], K))

# グラフの描画
fig, ax = plt.subplots()
cmap = ['Blues', 'Oranges', 'Greens']
for k in range(K):
    ax.scatter(moonX[moonY==k, 0], moonX[moonY==k, 1])
    ax.contourf(x_mesh, y_mesh, pp[:, :, k], levels=[0.5, 0.6, 0.7, 0.8, 0.9, 1.0], cmap=cmap[k], alpha=0.3)
ax.set_xlim(xmin, xmax)
ax.set_ylim(ymin, ymax)
ax.set_aspect('equal')
plt.show()

2層や3層のニューラルネットワークでは，2クラスの境界複雑な曲線を描くことでこのデータをうまく識別できていることが分かります．